# Инструкции к решению задачи
Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным 0.05.

Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)

для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [1]:
import pandas as pd
import numpy as np
'''
patient id1    diagnosis    gene1_expr    gene2_expr ........
patient id2    diagnosis    gene1_expr    gene2_expr ........
. . . . . .. . . . . . . . . . . . . . . . . . ..  . . .. . . 
'''
data = pd.read_csv('C:\\Users\\user\\Desktop\\genes.csv', sep=",", header=0)
print(data.columns)

Index(['Patient_id', 'Diagnosis', 'LOC643837', 'LOC100130417', 'SAMD11',
       'NOC2L', 'KLHL17', 'PLEKHN1', 'C1orf170', 'HES4',
       ...
       'CLIC2', 'RPS4Y1', 'ZFY', 'PRKY', 'USP9Y', 'DDX3Y', 'CD24', 'CYorf15B',
       'KDM5D', 'EIF1AY'],
      dtype='object', length=15750)


In [8]:
import scipy
normal = data[data.Diagnosis == 'normal']
early_neoplasia = data[data.Diagnosis == 'early neoplasia']
cancer = data[data.Diagnosis == 'cancer']
#совпадают ли списки генов для разных категорий? нет ли артефактов?
normal.columns.tolist() == early_neoplasia.columns.tolist()

True

In [9]:
cancer.columns.tolist() == early_neoplasia.columns.tolist()

True

In [17]:
column_list = cancer.columns.tolist()[2:] # список названй генов
# тест дает (статистика, р val)
ttest_ind1 = [scipy.stats.ttest_ind(early_neoplasia[column], normal[column], equal_var = False) for column in column_list]

mean_gene_exp_normal = [np.mean( normal[column]) for column in column_list] 
mean_gene_exp_early = [np.mean( early_neoplasia[column]) for column in column_list] 
mean_gene_exp_cancer = [np.mean( cancer[column]) for column in column_list]

print (len([test.pvalue for test in ttest_ind1 if test.pvalue < 0.05]))

1575


In [11]:
ttest_ind2 = [scipy.stats.ttest_ind(early_neoplasia[column], cancer[column], equal_var = False) for column in column_list]
print (len([test.pvalue for test in ttest_ind2 if test.pvalue < 0.05]))

3490


В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [12]:
def write_answer(task,ans):
    with open("C:\\Users\\user\\Desktop\\" + 'bio_' + str(task)+".txt", "w") as fout:
        fout.write(str(ans))
write_answer(1,1575)
write_answer(2,3490)

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5

Практическая значимость изменения
Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:f = a/b (if a > b) or -b/a (if b > a). По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

In [13]:
import statsmodels.stats.multitest as smm
norm_early_pvals = [test.pvalue for test in ttest_ind1]
cans_early_pvals = [test.pvalue for test in ttest_ind2]
holm_norm_early = smm.multipletests(norm_early_pvals, alpha=0.025, method='holm', is_sorted=False, returnsorted=False)
cans_norm_early = smm.multipletests(cans_early_pvals, alpha=0.025, method='holm', is_sorted=False, returnsorted=False)
print(holm_norm_early)

(array([False, False, False, ..., False, False, False]), array([1.        , 0.50017368, 1.        , ..., 1.        , 1.        ,
       1.        ]), 1.6076827300537389e-06, 1.58750317500635e-06)


In [23]:
norm_early_H1 = len([x for x in holm_norm_early[0] if x == True])
print('holm corrected pvals=', norm_early_H1)
norm_early_H1_practice = len([y/z for x,y,z in zip(holm_norm_early[0], mean_gene_exp_normal, mean_gene_exp_early) 
                              if x == True and (y/z > 1.5 or z/y > 1.5)])
print('holm corrected pvals practical=', norm_early_H1_practice)

holm corrected pvals= 2
holm corrected pvals practical= 2


In [27]:
cans_early_H1 = len([x for x in cans_norm_early[0] if x == True])
print('holm corrected pvals=', cans_early_H1)
cancer_early_H1_practice = len([y/z for x,y,z in zip(cans_norm_early[0], mean_gene_exp_cancer, mean_gene_exp_early) 
                                if x == True and (y/z > 1.5 or z/y > 1.5)])
print('holm corrected pvals practical=', cancer_early_H1_practice)

holm corrected pvals= 79
holm corrected pvals practical= 77


In [29]:
write_answer(3,2)
write_answer(4,77)

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H_0, когда они есть, и будут чаще отклонять H_0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [30]:
import statsmodels.stats.multitest as smm
norm_early_pvals = [test.pvalue for test in ttest_ind1]
cans_early_pvals = [test.pvalue for test in ttest_ind2]
holm_norm_early = smm.multipletests(norm_early_pvals, alpha=0.025, method='fdr_bh', is_sorted=False, returnsorted=False)
cans_norm_early = smm.multipletests(cans_early_pvals, alpha=0.025, method='fdr_bh', is_sorted=False, returnsorted=False)
print(holm_norm_early)

norm_early_H1 = len([x for x in holm_norm_early[0] if x == True])
print('holm corrected pvals=', norm_early_H1)
norm_early_H1_practice = len([y/z for x,y,z in zip(holm_norm_early[0], mean_gene_exp_normal, mean_gene_exp_early) 
                              if x == True and (y/z > 1.5 or z/y > 1.5)])
print('holm corrected pvals practical=', norm_early_H1_practice)

cans_early_H1 = len([x for x in cans_norm_early[0] if x == True])
print('holm corrected pvals=', cans_early_H1)
cancer_early_H1_practice = len([y/z for x,y,z in zip(cans_norm_early[0], mean_gene_exp_cancer, mean_gene_exp_early) 
                                if x == True and (y/z > 1.5 or z/y > 1.5)])
print('holm corrected pvals practical=', cancer_early_H1_practice)

(array([False, False, False, ..., False, False, False]), array([0.96651052, 0.03569758, 0.53610276, ..., 0.96312033, 0.97735971,
       0.96074074]), 1.6076827300537389e-06, 1.58750317500635e-06)
holm corrected pvals= 4
holm corrected pvals practical= 4
holm corrected pvals= 832
holm corrected pvals practical= 524


In [31]:
write_answer(5,4)
write_answer(6,524)